This notebook is for snapping of substations to lines

In [ ]:
import os
import matplotlib.pyplot as plt 
import geopandas as gpd
import geoplot
import pandas as pd
import numpy as np
import pandas as pd
import hvplot.pandas

import sys
sys.path.append('../')  # to import helpers
from scripts._helpers import _sets_path_to_root
_sets_path_to_root("pypsa-africa")

# Africa shape data

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
africa = world.query('continent == "Africa"')
nigeriaplot = world.query('name == "Nigeria"')
#world

# Power data

In [ ]:
pathg = os.path.realpath("data/raw")+'/africa_all_raw_generators.geojson' # Generators are not required in base_network
paths = os.path.realpath("data/clean")+'/africa_all_buses_clean.geojson'
pathl = os.path.realpath("data/clean")+'/africa_all_lines_clean.geojson'

generators = gpd.read_file(pathg).set_crs(epsg=4326, inplace=True)
substations = gpd.read_file(paths).set_crs(epsg=4326, inplace=True)
lines = gpd.read_file(pathl).set_crs(epsg=4326, inplace=True)

In [ ]:
lines_ng = lines[lines["country"] == "nigeria"]
substations_ng = substations[substations["country"] == "nigeria"]
generators_ng = generators[generators["Country"] == "nigeria"]

In [ ]:
# Redact data frames
#g_ = generators[['id', 'tags.power', 'geometry', 'Type', 'Country']]
b_ = substations[['bus_id', 'symbol', 'geometry', 'country']]
l_ = lines[['line_id', 'tag_type', 'geometry', 'country']] #lines
l_.rename(columns={"line_id": "id"}, inplace="true")

In [ ]:
# NIGERIA AS TEST
b_ = b_[b_["country"] == "nigeria"]
l_ = l_[l_["country"] == "nigeria"]


In [ ]:
b_

In [ ]:
# https://gis.stackexchange.com/questions/48949/epsg-3857-or-4326-for-googlemaps-openstreetmap-and-leaflet
# EPSG 3857 for google maps projected coors

b_.to_crs("EPSG:3857", inplace=True) # Projecting from ESPG 4326 to 3857

pd.set_option('mode.chained_assignment',None) # Disable SetCopyWarning
l_.to_crs("EPSG:3857", inplace=True)
pd.reset_option("mode.chained_assignment") # Enable SetCopyWarning

In [ ]:
b_.bounds # Get Non-Existant Bounding Box of Points
# (minx, miny, maxx, maxy) = (west, south, east, north) = 
# west: Lower-left X coordinate
# south: Lower-left Y coordinate
# east: Upper-right X coordinate
# north: Upper-right Y coordinate

In [ ]:
offset = 100
bbox = b_.bounds + [-offset, -offset, offset, offset]
bbox # bbox is a square with length 200m

In [ ]:
l_

In [ ]:
hits = bbox.apply(lambda row: list(l_.sindex.intersection(row)), axis=1) # list of the lines that overlap that bounding box.
hits # Index of lines are not the Ids

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(pd.DataFrame.from_records([hits.apply(len).value_counts().to_dict()]).transpose())

In [ ]:
# Flatten Hits
tmp = pd.DataFrame({
    # index of points table
    "pt_idx": np.repeat(hits.index, hits.apply(len)),
    # ordinal position of line - access via iloc later
    "line_i": np.concatenate(hits.values)
})
tmp

In [ ]:
tmp = tmp.join(l_.reset_index(drop=True), on="line_i")

tmp = tmp.join(b_.geometry.rename("point"), on="pt_idx")

# tmp = tmp.join(b_.reset_index(drop=True), on="pt_idx")

tmp = gpd.GeoDataFrame(tmp, geometry="geometry", crs=b_.crs)
tmp

In [ ]:
tmp

In [ ]:
# TODO : Convert to Google Maps CRS (2D)
tmp["snap_dist"] = tmp.geometry.distance(gpd.GeoSeries(tmp.point))

In [ ]:
tmp

In [ ]:
tolerance = offset
# Discard any lines that are greater than tolerance from points
tmp = tmp.loc[tmp.snap_dist <= tolerance]
# Sort on ascending snap distance, so that closest goes to top
tmp = tmp.sort_values(by=["snap_dist"])



In [ ]:
# group by the index of the points and take the first, which is the
# closest line 
closest = tmp.groupby("pt_idx").first()
# construct a GeoDataFrame of the closest lines
closest = gpd.GeoDataFrame(closest, geometry="geometry")

In [ ]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    # display(closest)

In [ ]:
# Position of nearest point from start of the line
pos = closest.geometry.project(gpd.GeoSeries(closest.point))
# Get new point location geometry
new_pts = closest.geometry.interpolate(pos)

In [ ]:
# pos

In [ ]:
# new_pts

In [ ]:
#Identify the columns we want to copy from the closest line to the point, such as a line ID.
line_columns = 'line_i'
# Create a new GeoDataFrame from the columns from the closest line and new point geometries (which will be called "geometries")
snapped = gpd.GeoDataFrame(
closest[line_columns],geometry=new_pts)

points = b_

# Join back to the original points:
updated_points = points.drop(columns=["geometry"]).join(snapped)
# You may want to drop any that didn't snap, if so:
updated_points = updated_points.dropna(subset=["geometry"])

In [ ]:
# updated_points.to_file('./updates_points.geojson', driver="GeoJSON")  # Generate GeoJson

# Map

In [ ]:
# ax = geoplot.polyplot(nigeriaplot ,edgecolor='grey', alpha = 0.5, figsize=(20, 20))

# points.loc[points['country'] == 'nigeria'].plot(figsize=(20, 20),markersize=10, color='orange',alpha = 0.3, ax=ax)
# updated_points.loc[updated_points['country'] == 'nigeria'].plot(figsize=(20, 20),markersize=10, color='green',alpha = 0.3, ax=ax)
# # generators.loc[generators['country'] == 'nigeria'].plot(figsize=(20, 20),markersize=25, color='blue',alpha = 0.3, ax=ax)
# lines.loc[lines['country'] == 'nigeria'].plot(figsize=(20, 20),markersize=25, color='red',alpha = 0.3, ax=ax)


# #plt.savefig('africa_transmission_and substations_110.png')

In [ ]:
# substation_df = pd.read_csv(
#     "./data/clean/africa_all_buses_clean.csv",
#     index_col=0
# )

# substation_df.hvplot.points(
#     'x',
#     'y',
#     geo=True,
#     size = substation_df["tags_area"]**(0.5)/10,
#     frame_height=500,
#     alpha=0.4,
#     tiles='CartoLight',
#     hover_cols=['bus_id'],
#     c='tags_country'
# ).opts(
#     active_tools=['pan', 'wheel_zoom']
# )

In [ ]:
substations_ng.hvplot(
    geo=True,
    size = substations["tag_area"]**(0.5)/10,
    frame_height=750,
    alpha=0.4,
    tiles='CartoLight',
    hover_cols=['bus_id'],
    color = 'orange'  
) * lines_ng.hvplot(
    geo=True,
    alpha=0.4,
    hover_cols=['line_id']
) * generators_ng.hvplot(
    geo=True,
    alpha=0.4,
    color = 'green'
).opts(
    active_tools=['pan', 'wheel_zoom']
)